# Capstone 2 Project

## NLP training on ca. 3 million Yelp reviews

## What is covered in this part of the project
This part of the project deals with natural language processing of the Yelp data set. Specifically, it will import the raw data in json format, filter the reviews that relate to restaurants, prepare the data for NLP and perform a number of NLP functions:

1. Text preparation, including tokenization, lemmatization and normalization of the text using the spaCy library
1. Phrase modeling using gensim
1. Topic modeling with LDA
1. Word vector models with word2vec
1. Clustering words that commonly appear together 

Topic modeling and word2vec clusters will provide context to the reviews in order to return summary information that is easily interpretable and understandable to an average user, and may provide useful insight.

## Loading the Yelp data
[**The Yelp Dataset**](https://www.yelp.com/dataset_challenge/) is a dataset published by the business review service [Yelp](http://yelp.com) for academic research and educational purposes. I really like the Yelp dataset as a subject for machine learning and natural language processing demos, because it's big (but not so big that you need your own data center to process it), well-connected, and anyone can relate to it &mdash; it's largely about food, after all!

**Note:** If you'd like to execute this notebook interactively on your local machine, you'll need to download your own copy of the Yelp dataset. 

After filtering for restaurants, there are approximately __52K__ restaurants with approximately __2.9M__ user reviews related to them.

The raw data is available in six of files in _.json_ format, of which two are relevant for the project:
- __business.json__ &mdash; _the records for individual businesses_
- __review.json__ &mdash; _the records for reviews users wrote about businesses_

The files are text files (UTF-8) with one _json object_ per line, each one corresponding to an individual data record. 

In [1]:
import os
import codecs

data_directory = os.path.join('data',
                              'dataset')

businesses_filepath = os.path.join(data_directory,
                                   'business.json')

with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print first_business_record

{"business_id": "YDf95gJZaq05wvo7hTQbbQ", "name": "Richmond Town Square", "neighborhood": "", "address": "691 Richmond Rd", "city": "Richmond Heights", "state": "OH", "postal_code": "44143", "latitude": 41.5417162, "longitude": -81.4931165, "stars": 2.0, "review_count": 17, "is_open": 1, "attributes": {"RestaurantsPriceRange2": 2, "BusinessParking": {"garage": false, "street": false, "validated": false, "lot": true, "valet": false}, "BikeParking": true, "WheelchairAccessible": true}, "categories": ["Shopping", "Shopping Centers"], "hours": {"Monday": "10:00-21:00", "Tuesday": "10:00-21:00", "Friday": "10:00-21:00", "Wednesday": "10:00-21:00", "Thursday": "10:00-21:00", "Sunday": "11:00-18:00", "Saturday": "10:00-21:00"}}



The business records consist of _key, value_ pairs containing information about the particular business. The information from this file is treated in a separate notebook named _"Restaurants"_. 

This notebook will process information contained in the _reviews.json_ file.

In [2]:
review_json_filepath = os.path.join(data_directory,
                                    'review.json')

with codecs.open(review_json_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
print first_review_record

{"review_id":"VfBHSwC5Vz_pbFluy07i9Q","user_id":"cjpdDjZyprfyDG3RlkVG3w","business_id":"uYHaNptLzDLoV_JZ_MuzUA","stars":5,"date":"2016-07-12","text":"My girlfriend and I stayed here for 3 nights and loved it. The location of this hotel and very decent price makes this an amazing deal. When you walk out the front door Scott Monument and Princes street are right in front of you, Edinburgh Castle and the Royal Mile is a 2 minute walk via a close right around the corner, and there are so many hidden gems nearby including Calton Hill and the newly opened Arches that made this location incredible.\n\nThe hotel itself was also very nice with a reasonably priced bar, very considerate staff, and small but comfortable rooms with excellent bathrooms and showers. Only two minor complaints are no telephones in room for room service (not a huge deal for us) and no AC in the room, but they have huge windows which can be fully opened. The staff were incredible though, letting us borrow umbrellas for t

A few attributes of note on the review records:
- __business\_id__ &mdash; _identifies the business in question and provides a link to each business in the business.json file_
- __text__ &mdash; _the actual text of the review written by the user_


Work required to prepare the text for NLP analysis requires the following:
1. Loading each business record line by line using _json.loads_. Python converts json objects into a Python dict
2. Filter each business record to include only those that have "Restaurant: in the category list
3. Form a set of the business_id records in order to use this filter when processing the reviews file

In [3]:
import json

restaurant_ids = set()

# open the businesses file
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a restaurant, skip to the next one
        if u'Restaurants' not in business[u'categories']:
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business[u'business_id'])

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
restaurant_ids = frozenset(restaurant_ids)

# print the number of unique restaurant ids in the dataset
print '{:,}'.format(len(restaurant_ids)), u'restaurants in the dataset.'

51,613 restaurants in the dataset.


This step creates a new file that contains only the text from reviews about restaurants, with one review per line in the file.

In [4]:
intermediate_directory = os.path.join('data', 'intermediate')

review_txt_filepath = os.path.join(intermediate_directory,
                                   'review_text_all.txt')

In [5]:
%%time

# this part is complete so make the statement false to skip directly to reading the file
if 0 == 1:
    
    review_count = 0

    # create & open a new file in write mode
    with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with codecs.open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant, skip to the next one
                if review[u'business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print u'''Text from {:,} restaurant reviews
              written to the new txt file.'''.format(review_count)
    
else:
    
    with codecs.open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print u'Text from {:,} restaurant reviews in the txt file.'.format(review_count + 1)

Text from 2,929,512 restaurant reviews in the txt file.
Wall time: 1min 26s


## Text preparation using spaCy

The [**spaCy**](https://spacy.io) library will be used to perform the following NLP tasks: 
- Tokenization
- Text normalization, including converting text to lemmatized form 
- Part-of-speech tagging
- Syntactic dependency parsing
- Sentence boundary detection
- Named entity recognition and annotation

spaCy provides downloadable English-language models that enable the comparison of a specific corpus of text to a general English language corpus, such as common stpowords, and the probability of occurence of words. 

In [6]:
import pandas as pd
import itertools as it

The next part is needed to append the system path list to access the location of the spaCy module and English language model on my laptop.

In [7]:
import sys
sys.path.append('C:\Windows\System32\.env\Lib\site-packages')

This part now handles the text processing using spaCy

In [8]:
import spacy
import en_core_web_sm
# English model will only load with this code, does not work using the code provided in spaCy documentation
nlp = en_core_web_sm.load() 

In [9]:
# this examines a sample review
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print sample_review

The staff here is great and they're nice,  wonderful and quick. People were ranting in raving about pei wei, I had to try it.  Even good yelp reviews.  I'm highly dissatisfied with the flavor of the food. This  should be labeled Asian inspired and not Asian. I've tried a variety of Chinese restaurants, this doesn't taste close to anything I've had at other Asian restaurants. Their Mongolian beef  was 5 pieces of beef and large mushrooms cut into thirds in a thick sauce. You eat the rice to wash off the nasty flavor. My shrimp was thickly coated in an overpowering  sauce as well.  I only ate some of the veggies that take center stage on a meat dish.  The center of my pork egg roll was cold. The hot N sour soup was a much thicker consistency almost like that of a chili instead of being brothy. Worst of all was the price.  This was not worth it to us. Neither me or my husband enjoyed either of  our dishes.  We didn't even eat half of our plates.  We even refused to take it home with us.  

Sentence detection and segmentation:

In [10]:
parsed_review = nlp(sample_review)

for num, sentence in enumerate(parsed_review.sents):
    print 'Sentence {}:'.format(num + 1)
    print sentence
    print ''

Sentence 1:
The staff here is great and they're nice,  wonderful and quick.

Sentence 2:
People were ranting in raving about pei wei, I had to try it.  

Sentence 3:
Even good yelp reviews.  

Sentence 4:
I'm highly dissatisfied with the flavor of the food.

Sentence 5:
This  should be labeled Asian inspired and not Asian.

Sentence 6:
I've tried a variety of Chinese restaurants, this doesn't taste close to anything I've had at other Asian restaurants.

Sentence 7:
Their Mongolian beef  was 5 pieces of beef and large mushrooms cut into thirds in a thick sauce.

Sentence 8:
You eat the rice to wash off the nasty flavor.

Sentence 9:
My shrimp was thickly coated in an overpowering  sauce as well.  

Sentence 10:
I only ate some of the veggies that take center stage on a meat dish.  

Sentence 11:
The center of my pork egg roll was cold.

Sentence 12:
The hot N sour soup was a much thicker consistency almost like that of a chili instead of being brothy.

Sentence 13:
Worst of all was the 

Named entity detection:

In [11]:
for num, entity in enumerate(parsed_review.ents):
    print 'Entity {}:'.format(num + 1), entity, '-', entity.label_
    print ''

Entity 1: Asian - NORP

Entity 2: Asian - NORP

Entity 3: Chinese - NORP

Entity 4: Asian - NORP

Entity 5: Mongolian - NORP

Entity 6: 5 - CARDINAL

Entity 7: half - CARDINAL

Entity 8: Asian - NORP



Part of speech tagging:

In [12]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(zip(token_text, token_pos),
             columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,The,DET
1,staff,NOUN
2,here,ADV
3,is,VERB
4,great,ADJ
5,and,CCONJ
6,they,PRON
7,'re,VERB
8,nice,ADJ
9,",",PUNCT


Text normalization includes converting words to their lemmatized form so that grammatical form is removed in order to simplify and reduce the vocabulary. Hence words are converted to lowercase and words with the same stem but different grammatical forms are converted to a single lemma (e.g. "be:, "is", "were", "am", "are" are all represented as "be" in lemmatized form).

In [13]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(zip(token_text, token_lemma, token_shape),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,The,the,Xxx
1,staff,staff,xxxx
2,here,here,xxxx
3,is,be,xx
4,great,great,xxxx
5,and,and,xxx
6,they,-PRON-,xxxx
7,'re,be,'xx
8,nice,nice,xxxx
9,",",",",","


## Phrase Modeling with gensim

_Phrase modeling_ is an algorithm that passes over the text corpus to learn multi-word concepts. Using the algorithm once will detect phrases such as "ice cream" by detecting that "ice" and "cream" appear together with a frequency that passes a certain threshold. At the next pass, the algorithm will treat the bigram "ice_cream" as a single token and may also learn "vanilla_ice_cream" as a phrase and treat it as a trigram. 

This part makes three passes over the text corpus in order to reveal trigrams, which will help provide a robust level of meaning, given that the corpus is fairly large.

In [14]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

C:\Users\Work\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using CNTK backend
C:\Users\Work\Anaconda2\lib\site-packages\keras\backend\cntk_backend.py:19: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


As we're performing phrase modeling, we'll be doing some iterative data transformation at the same time. Our roadmap for data preparation includes:

1. Segment text of complete reviews into sentences & normalize text
1. First-order phrase modeling $\rightarrow$ _apply first-order phrase model to transform sentences_
1. Second-order phrase modeling $\rightarrow$ _apply second-order phrase model to transform sentences_
1. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

In [15]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [16]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')

The `lemmatized_sentence_corpus` generator loops over the original review text, segmenting the reviews into individual sentences and normalizing the text. The output is written to a new file (`unigram_sentences_all`), with one normalized sentence per line.

In [17]:
%%time

# this is completed

if 0 == 1:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')

Wall time: 0 ns


In [18]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

The next phase passes over the text to link individual words into two-word phrases e.g. linking "`ice cream`" into a new, single token: "`ice_cream`".

In [19]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [20]:
%%time

# this is completed

if 0 == 1:

    bigram_model = Phrases(unigram_sentences, max_vocab_size=10000000)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 3.47 s


In [21]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')

In [22]:
%%time

# this is completed
# warning message: For a faster implementation, use the gensim.models.phrases.Phraser class
if 0 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

Wall time: 0 ns


In [23]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [24]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

In [25]:
%%time

# this is completed
if 0 == 1:

    trigram_model = Phrases(bigram_sentences, max_vocab_size=10000000)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 4.98 s


In [26]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')

In [27]:
%%time

# this is completed
if 0 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

Wall time: 0 ns


In [28]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

The next step comprises the following: 
- running the complete text of the reviews through a pipeline that applies text normalization and phrase models
- removing stopwords
- writing the output to a new file with one review per line


In [29]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [30]:
%%time

# this is completed
if 0 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.language_data.STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

Wall time: 0 ns


## Topic Modeling with Latent Dirichlet Allocation (_LDA_)

In [31]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import warnings
import cPickle as pickle

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [33]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [34]:
%%time

# this is completed
if 0 == 1:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 69 ms


In [35]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [36]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [37]:
%%time

# this is completed
if 0 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 971 ms


In [38]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [39]:
%%time

# this is completed
if 0 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

Wall time: 344 ms


In [40]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print u'{:20} {}'.format(u'term', u'frequency') + u'\n'

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print u'{:20} {:.3f}'.format(term, round(frequency, 3))

In [41]:
explore_topic(topic_number=49)

term                 frequency

review               0.062
yelp                 0.023
know                 0.020
read                 0.020
write                0.014
guy                  0.011
people               0.010
like                 0.010
man                  0.009
let                  0.009
think                0.009
u                    0.008
want                 0.008
's                   0.007
post                 0.007
groupon              0.007
star                 0.006
use                  0.006
mean                 0.005
base                 0.005
yes                  0.005
way                  0.005
question             0.005
update               0.004
girl                 0.004


With the text corpus grouped into clusters, we can explore the words in each cluster to identify a common theme/topic. The list below attempts to identify a common-sense title for each cluster. We can then use this to summarize a review in terms of 3-5 key topics discussed by the reviewer. 

In [42]:
topic_names = {0: u'dessert',
               1: u'table, ordering & service',
               2: u'airport & flying, allergies',
               3: u'buffet & dishes',
               4: u'pizza & italian',
               5: u'bar & drinking',
               6: u'salad & lunch',
               7: u'kids & family',
               8: u'las vegas',
               9: u'fish & asian',
               10: u'seating experience',
               11: u'waffle / cupcake', # waffle + various unrelated terms
               12: u'family members',
               13: u'flavor & food experience',
               14: u'general ambience',
               15: u'night / happy hour / drinks',
               16: u'brunch',
               17: u'mexican',
               18: u'meat',
               19: u'price & paying',
               20: u'wedding / german', # various unrelated words incl single letters
               21: u'various locations', # e.g. north, south, town names, uptown, street etc.
               22: u'cleansiness',
               23: u'mall / cheesecake_factory', # plus various unrelated words and some experience words
               24: u'positive service experience',
               25: u'buying / offers',
               26: u'positive food experience',
               27: u'cheese / wings',
               28: u'french language reviews',
               29: u'positive experience',
               30: u'ordering and service', # geenral and negative service experience
               31: u'time & place',
               32: u'menu & dishes',
               33: u'sushi & fish',
               34: u'german language reviews',
               35: u'positive food experience',
               36: u'timing & waiting',
               37: u'seafood',
               38: u'tea & drinks',
               39: u'thai',
               40: u'staff', 
               41: u'sandwiches',
               42: u'yes / know / eat', # unrelated common words
               43: u'hot_dog, chilli', #hot is the most prominent word, both in teh sense of spicy and hot_dog
               44: u'drinks / coffee / atmosphere',
               45: u'burgers',
               46: u'breakfast food', 
               47: u'price / service / portions',
               48: u'steaks',
               49: u'review / yelp'} # various common urelated words

In [43]:
topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'w') as f:
    pickle.dump(topic_names, f)

There are a variety of topics related to different types of restaurant, such as **pizza&italian**, **mexican**, **thai**, **sushi&fish**, etc., as well as topics related to the restaurant experience such as seating, service and pricing. 

Beyond these two categories, there are still some topics that are not straightforward to interpret, such as topics 23 and 42.


## Word Vector Embedding with Word2Vec

The goal of *word vector embedding models*, or *word vector models* for short, is to learn dense, numerical vector representations for each term in a corpus vocabulary. If the model is successful, the vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary. Word vector models are also fully unsupervised &mdash; they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.

Here we will use the _gensim_ [word2vec](https://arxiv.org/pdf/1301.3781v3.pdf) model originally proposed in 2013. The general idea of word2vec is, for a given *focus word*, to use the *context* of the word &mdash; i.e., the other words immediately before and after it &mdash; to provide hints about what the focus word might mean. To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.

One complete pass sliding the window across all of the corpus text is known as a training *epoch*. We will train the model over multiple passes/epochs in order to optimize the vector representations that are *close* to each other in vector space.

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors - we will use 100-dimensional vector space to allow sufficient complexity at reasonable computing cost
- The width of the sliding window, in tokens - default is used
- The number of training epochs - we will use 5 epochs


In [44]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

This section trains the word2vec model using the normalized sentences of the tri-gram phrase model in 100-dimensional vector space, using 5 epochs.

In [45]:
if 0 ==1:
    food2vec.train(trigram_sentences, total_examples=food2vec.corpus_count, epochs=food2vec.iter)
    print food2vec.train_count
    food2vec.save(word2vec_filepath)
    
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print u'{} training epochs so far.'.format(food2vec.train_count)

2 training epochs so far.


In [46]:
print food2vec.iter
print food2vec.train_count

5
2


In [47]:
print u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.wv.vocab))

80,588 terms in the food2vec vocabulary.


To look at the word vector model, we can create a pandas DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [48]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.wv.vocab.iteritems()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda (term, index, count): -count)

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
-PRON-,-0.075121,-0.067430,-0.127061,-0.009218,-0.022045,0.170232,0.160691,0.043578,0.201154,-0.037751,...,-0.080210,-0.049757,0.025534,0.015274,-0.086829,-0.124214,0.064420,0.040135,-0.064753,0.124752
be,0.012676,-0.047068,-0.190982,-0.033070,0.053919,0.250453,0.093323,-0.045521,0.229455,0.075257,...,-0.169080,-0.003657,-0.147224,-0.002735,-0.115380,-0.210662,0.005098,0.041456,0.005519,-0.022719
the,0.114931,0.122418,-0.168722,0.071732,-0.059859,0.315316,0.036166,-0.029906,0.292869,-0.101682,...,-0.023712,-0.109440,-0.081754,0.105876,-0.073468,-0.165876,0.151434,0.016965,-0.156994,0.009750
and,0.114759,-0.138922,-0.144100,0.006292,0.117541,0.288687,0.099160,0.027657,0.343955,0.049567,...,-0.030788,-0.053569,-0.044293,0.042094,-0.086180,-0.017890,0.075092,-0.055175,-0.031652,0.033878
a,-0.032536,-0.074984,0.064469,-0.171038,0.020188,0.134144,0.109164,0.008352,0.244023,0.021498,...,-0.041942,0.023825,-0.163675,0.137406,-0.065831,-0.101458,0.000391,-0.006488,-0.038800,0.170717
to,0.040521,0.001736,-0.043932,0.044770,-0.113628,-0.022404,0.098611,-0.007043,0.223237,-0.003608,...,-0.111642,-0.035838,0.109139,0.119915,-0.073793,-0.023686,-0.009697,-0.005111,-0.104554,0.023787
have,-0.084267,-0.004397,-0.139282,-0.078336,0.152177,0.199497,-0.005634,0.043972,0.179384,-0.006454,...,-0.177000,0.035325,-0.133072,0.059747,-0.014941,0.053005,0.155703,-0.023206,-0.083596,-0.067215
of,0.032294,-0.078110,-0.054774,0.063565,0.002737,0.204627,0.098279,0.024468,0.223266,0.041319,...,-0.088307,0.018960,0.040729,0.075636,-0.002357,-0.060602,0.052249,-0.035362,-0.188302,0.044007
not,-0.052620,-0.045803,-0.088368,0.092387,-0.012385,-0.009546,0.076239,0.026310,0.149709,0.072366,...,-0.033276,-0.031341,0.009931,0.066965,-0.087295,-0.164688,0.060534,0.188567,0.038080,0.041429
for,0.108099,0.026058,0.099891,-0.059244,-0.024895,0.214824,0.016041,-0.035647,0.049735,-0.035197,...,-0.053567,-0.012096,-0.098778,0.110779,-0.126084,-0.069208,0.050522,0.049058,-0.100268,0.079133


In [49]:
word_vectors.shape

(80588, 100)

This DataFrame has 80,588 rows &mdash; one for each term in the vocabulary &mdash; and 100 colums. This is the vector representation for each term.

Next, we will save this as a gensim model and it can be reused in another notebook for converting any text into the 100-dimensional vector representation. 

This will provide more _meaning_ to a review, compared to the LDA topic modeling. The word2vec can provide information about what the text is about and if the words have a positive-negative context. The LDA only provides information about the topics (sometimes hard to interpret), but very little information about the positive/negative sentiment of the review. 